In [1]:
from pysymphony import SymphonyClient
from dotenv import load_dotenv
import logging

logging.basicConfig(level=logging.INFO)
load_dotenv()

client = SymphonyClient()

INFO:root:GET http://127.0.0.1:8000/inference/tools/list: RUNNING
INFO:root:GET http://127.0.0.1:8000/inference/tools/list: SUCCESS


In [2]:
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>}]

In [3]:
from pysymphony.settings import settings

def add_tool(a: int, b: int):
    """
    Add two numbers
    """
    print(f"`add_tool` tool called on the client side with inputs {a} and {b}")
    return a + b

settings.add_local_tool(add_tool)

INFO:root:POST http://127.0.0.1:8000/inference/tools/add: SUCCESS


In [4]:
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'add_tool',
  'name': 'add_tool',
  'descri

In [5]:
from pysymphony.workflows import Workflow

task = (
    "Given a list of numbers, return the sum of the numbers."
)
parameters = {
    "max_steps": 1
}
useful_tools = ["add_tool"]
workflow = client.generate_workflow(task, parameters, useful_tools)
print(workflow)

INFO:root:POST http://127.0.0.1:8000/inference/planner/generateworkflow: SUCCESS
INFO:root:GET http://127.0.0.1:8000/inference/planner/getworkflows: RUNNING
INFO:root:GET http://127.0.0.1:8000/inference/planner/getworkflows: SUCCESS


Workflow(id=e9e384de-e186-4564-bbdb-7fa9d832350f, task_description=Given a list of numbers, return the sum of the numbers.)


In [6]:
run = workflow.init_run(input="12312413312312, 12312312312, 12312312, 123123")

INFO:root:POST http://127.0.0.1:8000/inference/runner/initrun: SUCCESS


In [7]:
run.run()

INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12312413312312 and 12312312312


INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12312413312312 and 12312312312
`add_tool` tool called on the client side with inputs 12324725624624 and 12312312


INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS


`add_tool` tool called on the client side with inputs 12312413312312 and 12312312312
`add_tool` tool called on the client side with inputs 12324725624624 and 12312312
`add_tool` tool called on the client side with inputs 12324737936936 and 123123


INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS


<Status.SUCCESS: 'success'>

In [8]:
run.ios

{'0': {'input': None,
  'output': '12312413312312, 12312312312, 12312312, 123123'},
 '1': {'input': 'A list of numbers: 12312413312312, 12312312312, 12312312, 123123',
  'output': 'The total sum of the numbers 12312413312312, 12312312312, 12312312, and 123123 is 12324738060059.'}}

In [9]:
from pysymphony.workflows import Workflow

task = (
    "Given a company name, find the competitors and the founder's background and output a json of the information."
)
parameters = {
    "max_steps": 3
}
useful_tools = ["pplx_online"]
workflow = client.generate_workflow(task, parameters, useful_tools)
print(workflow)

INFO:root:POST http://127.0.0.1:8000/inference/planner/generateworkflow: SUCCESS


Workflow(id=0cbc3b5e-753f-4779-bb5c-4c9e952b0009, task_description=Given a company name, find the competitors and the founder's background and output a json of the information.)


In [10]:
run = workflow.init_run(input="Coinbase")
run.run()
run.ios

INFO:root:POST http://127.0.0.1:8000/inference/runner/initrun: SUCCESS
INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS
INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS
INFO:root:POST http://127.0.0.1:8000/inference/runner/runstep: SUCCESS


{'0': {'input': None, 'output': 'Coinbase'},
 '1': {'input': 'Company name: Coinbase',
  'output': '```json\n{\n  "company": "Coinbase",\n  "competitors": [\n    {\n      "name": "Binance",\n      "description": "A global giant in cryptocurrency trading, offering a diverse range of cryptocurrencies and low fees."\n    },\n    {\n      "name": "Kraken",\n      "description": "One of the largest Bitcoin exchanges in terms of euro volume and liquidity, supporting multiple fiat currencies."\n    },\n    {\n      "name": "Gemini",\n      "description": "Prioritizes security and regulatory compliance, making it a reliable choice for users."\n    },\n    {\n      "name": "KuCoin",\n      "description": "A centralized cryptocurrency exchange serving international markets with a wide range of digital assets."\n    },\n    {\n      "name": "LocalBitcoins",\n      "description": "A person-to-person bitcoin trading site, allowing users to trade cryptocurrencies directly with each other."\n    },\n

In [11]:
from IPython.display import Markdown

Markdown(run.ios["3"]['output'])

Here is the structured JSON format containing the competitors of Coinbase and detailed background information about its founder, Brian Armstrong:

```json
{
  "competitors": [
    {
      "name": "Binance",
      "description": "A global giant in cryptocurrency trading, offering a diverse range of cryptocurrencies and low fees."
    },
    {
      "name": "Kraken",
      "description": "One of the largest Bitcoin exchanges in terms of euro volume and liquidity, supporting multiple fiat currencies."
    },
    {
      "name": "Gemini",
      "description": "Prioritizes security and regulatory compliance, making it a reliable choice for users."
    },
    {
      "name": "KuCoin",
      "description": "A centralized cryptocurrency exchange serving international markets with a wide range of digital assets."
    },
    {
      "name": "LocalBitcoins",
      "description": "A person-to-person bitcoin trading site, allowing users to trade cryptocurrencies directly with each other."
    },
    {
      "name": "NiceHash",
      "description": "Operates on the concept of a sharing economy, connecting sellers and buyers of computing power."
    },
    {
      "name": "Bittrex",
      "description": "A U.S.-based blockchain platform providing real-time trading and industry-leading security."
    },
    {
      "name": "eToro",
      "description": "Excels in social trading features, allowing users to replicate the strategies of proven traders."
    },
    {
      "name": "Poloniex",
      "description": "A leading cryptocurrency exchange offering a wide variety of digital assets for trading."
    },
    {
      "name": "Blockchain",
      "description": "Specializes in blockchain technology and cryptocurrency-related solutions."
    },
    {
      "name": "Circle",
      "description": "Provides financial technology solutions for global fund transfers and crypto treasury management."
    },
    {
      "name": "BlockFi",
      "description": "Focuses on building products for cryptocurrencies, including interest-earning accounts and loan security."
    },
    {
      "name": "CoinZoom",
      "description": "Offers a platform for buying, selling, and trading cryptocurrencies with a focus on user experience."
    },
    {
      "name": "CoinDCX",
      "description": "Provides a cryptocurrency investment platform with a focus on user experience."
    },
    {
      "name": "FTX",
      "description": "Operates as a cryptocurrency exchange platform offering derivatives, options, and volatility products."
    },
    {
      "name": "BitPay",
      "description": "Develops software tools for businesses to collect payments through peer-to-peer payment networks."
    }
  ],
  "founder_background": {
    "name": "Brian Armstrong",
    "birth_date": "January 25, 1983",
    "education": {
      "undergraduate": {
        "degree": "Bachelor's in Economics and Computer Science",
        "institution": "Rice University",
        "year": 2005
      },
      "graduate": {
        "degree": "Master's in Computer Science",
        "institution": "Rice University",
        "year": 2006
      }
    },
    "previous_work_experience": [
      {
        "company": "IBM",
        "role": "Developer Intern"
      },
      {
        "company": "Deloitte",
        "role": "Consultant in Enterprise Risk Management"
      },
      {
        "company": "Airbnb",
        "role": "Software Engineer"
      }
    ],
    "notable_achievements": [
      {
        "achievement": "Founded Coinbase in 2012",
        "details": "Received $150,000 funding from Y Combinator."
      },
      {
        "achievement": "Launched GiveCrypto.org in 2018",
        "details": "A nonprofit aimed at distributing cryptocurrency globally."
      },
      {
        "achievement": "Signed the Giving Pledge in 2018",
        "details": "Committed to giving away the bulk of his wealth to charitable causes."
      },
      {
        "achievement": "Led Coinbase to a public listing on NASDAQ in 2021",
        "details": "Significant growth and recognition in the cryptocurrency industry."
      }
    ],
    "net_worth": "$7.4 billion (as of 2024)"
  }
}
```